# Scraping Movie Dates, Theater Data, and Award Data 

In this notebook, I'll describe the process used to web scrape movie information from three websites. The websites we will be using are:

1. https://www.moviefone.com/movies/2019/?page=1
2. https://www.boxofficemojo.com/chart/most_theaters/?by_studio_type=major
3. https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films

We will do some cleaning after we've scraped the data and then save the dataframes to csv files so that they can be used for later analysis.

First let's import some of the libraries we will need into the notebook.

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime as dt
import time

## Let's start by scraping from the moviefone.com website.

In [2]:
#Let's use this website to get specific movie release dates: https://www.moviefone.com/movies/2019/?page=1
#We're going to scrape the first page as a test to make sure we get the correct data.
#Then we will write a function to get movie release dates for a several years
#From this website we will look to obtain the movie title and the release date

movie_dates_page = requests.get("https://www.moviefone.com/movies/2019/?page=1")
soup = BeautifulSoup(movie_dates_page.content,'lxml')
movie_title = soup.find_all("a", class_="hub-movie-title")
movie_title 

[<a class="hub-movie-title" href="https://www.moviefone.com/movie/joker/PJ989VtkhszN0eNd45kPG6/main/" title="Joker">Joker</a>,
 <a class="hub-movie-title" href="https://www.moviefone.com/movie/how-to-train-your-dragon-the-hidden-world/20085119/main/" title="How to Train Your Dragon: The Hidden World">How to Train Your Dragon: The Hidden World</a>,
 <a class="hub-movie-title" href="https://www.moviefone.com/movie/parasite/xfuCgnN6ju51U5fVunSok/main/" title="Parasite">Parasite</a>,
 <a class="hub-movie-title" href="https://www.moviefone.com/movie/red-shoes-and-the-seven-dwarfs/FdhikQTnWHOSRwxxmXqhs2/main/" title="Red Shoes and the Seven Dwarfs">Red Shoes and the Seven Dwarfs</a>,
 <a class="hub-movie-title" href="https://www.moviefone.com/movie/terminator-dark-fate/v5h3H9xJn82zZumfdysg06/main/" title="Terminator: Dark Fate">Terminator: Dark Fate</a>,
 <a class="hub-movie-title" href="https://www.moviefone.com/movie/knives-out/3tdfGwebtBr2vXoaAE0x46/main/" title="Knives Out">Knives Out</a

Now we need to extract the title text from the HTML and we will want to store that information in a list. 

In [3]:
titles_list = [movie_title[i].text for i in range(0,len(movie_title))] 
titles_list #preview the list

['Joker',
 'How to Train Your Dragon: The Hidden World',
 'Parasite',
 'Red Shoes and the Seven Dwarfs',
 'Terminator: Dark Fate',
 'Knives Out',
 '1917',
 'Jumanji: The Next Level',
 'Marriage Story',
 "Darlin'",
 'Doctor Sleep',
 'Captain Marvel',
 'Jojo Rabbit',
 'Alita: Battle Angel',
 'Rambo: Last Blood',
 'Aladdin',
 'Dark Phoenix',
 'Godzilla: King of the Monsters',
 'Angel Has Fallen',
 '21 Bridges',
 'The Vast of Night',
 'It Chapter Two',
 'Zombieland: Double Tap',
 'After']

We'll use the same process to extract the dates for each movie as well.

In [4]:
movie_dates = soup.find_all("span", class_="hub-movie-date")
movie_date_list = [movie_dates[i].text for i in range(0, len(movie_dates))]
movie_date_list

['October 4, 2019',
 'February 22, 2019',
 'August 30, 2019',
 'August 1, 2019',
 'November 1, 2019',
 'November 27, 2019',
 'December 25, 2019',
 'December 13, 2019',
 'October 4, 2019',
 'March 9, 2019',
 'November 8, 2019',
 'March 8, 2019',
 'October 18, 2019',
 'February 14, 2019',
 'September 20, 2019',
 'May 24, 2019',
 'June 7, 2019',
 'May 31, 2019',
 'August 23, 2019',
 'November 22, 2019',
 'January 26, 2019',
 'September 6, 2019',
 'October 18, 2019',
 'April 12, 2019']

In [5]:
#Combine lists into a dataframe, rename columns, and preview the dataframe
release_dates_df = pd.DataFrame([titles_list, movie_date_list]).transpose()
release_dates_df.rename(columns={0:'movie', 1:'release_date'}, inplace=True)
release_dates_df.head()

,movie,release_date
0,Joker,"October 4, 2019"
1,How to Train Your Dragon: The Hidden World,"February 22, 2019"
2,Parasite,"August 30, 2019"
3,Red Shoes and the Seven Dwarfs,"August 1, 2019"
4,Terminator: Dark Fate,"November 1, 2019"




The release dates will be very useful, however we can can observe from the `movie_date_list` variable that each date is currently stored as a string.  We can use the datetime library to convert each of those strings into a datetime object. Let's see an example of this below:

In [6]:
date_time_obj = dt.datetime.strptime(release_dates_df['release_date'].iloc[0], '%B %d, %Y')
print(date_time_obj.date())

2019-10-04


In [10]:
#Let's convert the release_date column from a string to a datetime object
release_dates_df['release_date'] = pd.to_datetime(release_dates_df['release_date'], format='%B %d, %Y')


In [11]:
#Let's make new columns where we can see the month, day of the week, and year the movie was released
release_dates_df['release_month'] = release_dates_df['release_date'].map(lambda x: x.strftime('%B'))
release_dates_df['release_day'] = release_dates_df['release_date'].map(lambda x: x.strftime('%A'))
release_dates_df['release_year'] = release_dates_df['release_date'].map(lambda x: x.strftime('%Y'))

In [14]:
#Convert release_year column to an integer
release_dates_df['release_year'] = release_dates_df['release_year'].astype(int)

In [19]:
release_dates_df.head()


,movie,release_date,release_month,release_day,release_year
0,Joker,2019-10-04,October,Friday,2019
1,How to Train Your Dragon: The Hidden World,2019-02-22,February,Friday,2019
2,Parasite,2019-08-30,August,Friday,2019
3,Red Shoes and the Seven Dwarfs,2019-08-01,August,Thursday,2019
4,Terminator: Dark Fate,2019-11-01,November,Friday,2019


The dataframe shown above is exactly what we want. Now we can write a function to scrape multiple pages of this website and construct a dataframe with thousands of movies and their respective release date information.

It would be impratical in this case to attempt to scrape every webpage for a given year. The sheer number of movies would cause greatly increase the excecution time required for the function to run. The website lists 24 movies per page, so we scrape the first 10 pages for each year which will give us a total of 240 movies per year. We will select an appropriate start year so that we can have enough data to establish trends. The for loop in the function will repeat the processes shown above to populate each of the empty lists. The function then creates a dataframe and creates additional columns using the datetime library to convert the release date into pertinent data.

In [34]:
movie_releases_df = pd.DataFrame()
def scrape_movie_dates(start_year):
    """This function takes in a start year as an argument and constructs a dataframe
       displaying movie title, release, date, release month, release day, and release year.
       start_year must be an integer"""
    titles = []
    release_dates = []
    page_range = range(1,11)  #this is the range of pages we to scrape from
    stop_year = 2019          #this is the last year of movie data that we want
    for i in range(start_year, stop_year+1):
        for j in page_range:
            url = f'https://www.moviefone.com/movies/{i}/?page={j}'
            movie_dates_page = requests.get(url)
            soup = BeautifulSoup(movie_dates_page.content,'lxml')
            movie_title = soup.find_all("a", class_="hub-movie-title")
            titles += [movie_title[a].text for a in range(0,len(movie_title))]
            movie_dates = soup.find_all("span", class_="hub-movie-date")
            release_dates += [movie_dates[b].text for b in range(0, len(movie_dates))]
            time.sleep(.5)  #0.5 second pause between each page to ensure the request is not flagged
            
    movie_releases_df = pd.DataFrame([titles, release_dates]).transpose()
    movie_releases_df.rename(columns={0:'movie', 1:'release_date'}, inplace=True)
    
    movie_releases_df['release_date'] = pd.to_datetime(movie_releases_df['release_date'], format='%B %d, %Y')
    movie_releases_df['release_month'] = movie_releases_df['release_date'].map(lambda z: z.strftime('%B'))
    movie_releases_df['release_day'] = movie_releases_df['release_date'].map(lambda z: z.strftime('%A'))
    movie_releases_df['release_year'] = movie_releases_df['release_date'].map(lambda z: z.strftime('%Y'))
    movie_releases_df['release_year'] = movie_releases_df['release_year'].astype(int)
    
    return movie_releases_df
            
            
            

In [35]:
#We'll use a start year of 1927 which will give us 92 years worth of data.
movie_releases_df = scrape_movie_dates(1927)

In [36]:
movie_releases_df.head()

,movie,release_date,release_month,release_day,release_year
0,Metropolis,1927-03-06,March,Sunday,1927
1,"Dr. Mabuse, the Gambler",1927-08-08,August,Monday,1927
2,The Unknown,1927-06-03,June,Friday,1927
3,The Jazz Singer,1927-10-06,October,Thursday,1927
4,Chicago,1927-12-23,December,Friday,1927


In [37]:
len(movie_releases_df)

22320

22,320 movies are in the dataframe!  This will provide plenty of data for analysis.
Let's save the dataframe as a csv file.

In [473]:
movie_releases_df.to_csv('movie_release_dates.csv')

## Scraping Theater Data

We are going to scrape the BoxOfficeMojo website to scrape data pertaining to the number of theaters, domestic gross, and studio associated with a movie.

In [46]:
#Let's get movies that were in the most theaters by movie studio
#We also will get some financial info
#Let's get the titles of these movies
theaters_page = requests.get("https://www.boxofficemojo.com/chart/most_theaters/?by_studio_type=major")
theater_soup = BeautifulSoup(theaters_page.content,'lxml')
title_container = theater_soup.find_all("td", class_="a-text-left mojo-field-type-release mojo-cell-wide")

#We need a container variable for since the title data is within nested html
#Then we can use the findChild method to extract the titles

theater_titles = [title_container[i].findChild() for i in range(0, len(title_container))]
theater_titles_list = [theater_titles[x].text for x in range(0, len(theater_titles))]
theater_titles_list

['The Lion King',
 'Avengers: Endgame',
 'Spider-Man: Far from Home',
 'Toy Story 4',
 'It Chapter Two',
 'The Secret Life of Pets 2',
 'Despicable Me 3',
 'Jurassic World: Fallen Kingdom',
 'Aladdin',
 'Avengers: Infinity War',
 'Frozen II',
 'Incredibles 2',
 'Star Wars: Episode IX - The Rise of Skywalker',
 'The Dark Knight Rises',
 'Mission: Impossible - Fallout',
 'Iron Man 2',
 'The Secret Life of Pets',
 'Solo: A Star Wars Story',
 'Harry Potter and the Deathly Hallows: Part 2',
 'Joker',
 'The Dark Knight',
 "Pirates of the Caribbean: At World's End",
 'The Avengers',
 'Deadpool 2',
 'Spider-Man: Homecoming',
 'Guardians of the Galaxy Vol. 2',
 'Fast & Furious Presents: Hobbs & Shaw',
 'The Fate of the Furious',
 'Spider-Man 3',
 'The Amazing Spider-Man 2',
 'The Amazing Spider-Man',
 'Minions',
 'Captain Marvel',
 'Onward',
 'Shazam!',
 'Finding Dory',
 'The Lego Movie 2: The Second Part',
 'Jurassic World',
 'How to Train Your Dragon: The Hidden World',
 'Harry Potter and the

In [50]:
#We are going to pull the value for the maximum number of theaters where a movie was showing.
max_theaters = theater_soup.find_all("td", class_="a-text-right mojo-field-type-positive_integer")
max_theater_list = [max_theaters[i].text for i in range(0, len(max_theaters))]
max_theater_list

['4,802',
 '4,662',
 '4,634',
 '4,575',
 '4,570',
 '4,564',
 '4,535',
 '4,485',
 '4,476',
 '4,474',
 '4,440',
 '4,410',
 '4,406',
 '4,404',
 '4,395',
 '4,390',
 '4,381',
 '4,381',
 '4,375',
 '4,374',
 '4,366',
 '4,362',
 '4,349',
 '4,349',
 '4,348',
 '4,347',
 '4,344',
 '4,329',
 '4,324',
 '4,324',
 '4,318',
 '4,311',
 '4,310',
 '4,310',
 '4,306',
 '4,305',
 '4,303',
 '4,291',
 '4,286',
 '4,285',
 '4,276',
 '4,276',
 '4,268',
 '4,267',
 '4,264',
 '4,259',
 '4,256',
 '4,256',
 '4,255',
 '4,253',
 '4,250',
 '4,248',
 '4,248',
 '4,248',
 '4,236',
 '4,234',
 '4,233',
 '4,232',
 '4,227',
 '4,226',
 '4,224',
 '4,223',
 '4,210',
 '4,207',
 '4,206',
 '4,198',
 '4,184',
 '4,175',
 '4,166',
 '4,165',
 '4,164',
 '4,164',
 '4,163',
 '4,163',
 '4,158',
 '4,157',
 '4,155',
 '4,154',
 '4,153',
 '4,151',
 '4,148',
 '4,145',
 '4,144',
 '4,144',
 '4,142',
 '4,141',
 '4,137',
 '4,134',
 '4,133',
 '4,132',
 '4,131',
 '4,130',
 '4,125',
 '4,118',
 '4,115',
 '4,115',
 '4,108',
 '4,102',
 '4,102',
 '4,101',


In [168]:
yr_container = theater_soup.find_all("td", class_="a-text-left mojo-field-type-date_interval")
years = [yr_container[i].findChild() for i in range(0, len(yr_container))]
year_released = [years[i].text[-4:] for i in range(0,len(years))]
#We select the last 4 elements in the yr_container as we only want to scrape the year from this webpage.
#We have a dataframe with movie dates already so we don't want to scrape the full release date again.
year_released

['2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2017',
 '2018',
 '2019',
 '2018',
 '2019',
 '2018',
 '2019',
 '2012',
 '2018',
 '2010',
 '2016',
 '2018',
 '2011',
 '2019',
 '2008',
 '2007',
 '2012',
 '2018',
 '2017',
 '2017',
 '2019',
 '2017',
 '2007',
 '2014',
 '2012',
 '2015',
 '2019',
 '2020',
 '2019',
 '2016',
 '2019',
 '2015',
 '2019',
 '2007',
 '2015',
 '2017',
 '2014',
 '2018',
 '2008',
 '2019',
 '2016',
 '2017',
 '2016',
 '2013',
 '2018',
 '2012',
 '2019',
 '2019',
 '2020',
 '2018',
 '2014',
 '2017',
 '2019',
 '2016',
 '2019',
 '2004',
 '2017',
 '2013',
 '2018',
 '2020',
 '2019',
 '2015',
 '2004',
 '2017',
 '2011',
 '2012',
 '2013',
 '2018',
 '2015',
 '2016',
 '2020',
 '2008',
 '2016',
 '2014',
 '2017',
 '2018',
 '2016',
 '2016',
 '2005',
 '2018',
 '2012',
 '2015',
 '2006',
 '2017',
 '2018',
 '2016',
 '2010',
 '2018',
 '2011',
 '2018',
 '2019',
 '2009',
 '2009',
 '2009',
 '2012',
 '2017',
 '2018',
 '2014',
 '2017',
 '2019',
 '2018',
 '2017',
 '2017',
 '2016',
 '2017',
 

Let's double check that each list we've compiled from BoxOfficeMojo is the same length.  When we are constructing our dataframe we don't want pandas to accidentally insert a null value for a row where we do in fact have data.

In [170]:
len(year_released) == len(max_theater_list) == len(theater_titles_list)

True

In [57]:
#Let's grab the domestic gross values from the website
total_dom_gross = theater_soup.find_all("td", class_="a-text-right mojo-field-type-money")
total_dom_list = [total_dom_gross[i].text for i in range(0,len(total_dom_gross))]
total_dom_list

['$543,638,043',
 '$858,373,000',
 '$390,532,085',
 '$434,038,008',
 '$211,593,228',
 '$158,874,395',
 '$264,624,300',
 '$417,719,760',
 '$355,559,216',
 '$678,815,482',
 '$477,373,578',
 '$608,581,744',
 '$515,202,542',
 '$448,139,099',
 '$220,159,104',
 '$312,433,331',
 '$368,384,330',
 '$213,767,512',
 '$381,011,219',
 '$335,451,311',
 '$533,345,358',
 '$309,420,425',
 '$623,357,910',
 '$318,491,426',
 '$334,201,140',
 '$389,813,101',
 '$173,956,935',
 '$226,008,385',
 '$336,530,303',
 '$202,853,933',
 '$262,030,663',
 '$336,045,770',
 '$426,829,839',
 '$61,555,145',
 '$140,371,656',
 '$486,295,561',
 '$105,806,508',
 '$652,270,625',
 '$160,799,505',
 '$292,004,738',
 '$459,005,868',
 '$172,558,876',
 '$177,002,924',
 '$167,510,016',
 '$317,101,119',
 '$114,766,307',
 '$330,360,194',
 '$152,901,115',
 '$325,100,054',
 '$409,013,994',
 '$213,515,506',
 '$179,020,854',
 '$144,105,346',
 '$60,716,390',
 '$84,158,461',
 '$137,690,172',
 '$245,439,076',
 '$620,181,382',
 '$316,831,246',


The code below follows the same generally structure used to store studio names in a list.  The slight difference is that we have to call the `.strip()` method on each text element to get rid of the newline (`\n`) element that occurs in some studio names.

In [59]:
studios = theater_soup.find_all("td", class_="a-text-left mojo-field-type-release_studios")
studio_list = [studios[i].text.strip() for i in range(0, len(studios))]
studio_list

['-',
 '-',
 '-',
 '-',
 'Warner Bros.',
 'Universal Pictures',
 'Universal Pictures',
 'Universal Pictures',
 '-',
 '-',
 '-',
 '-',
 '-',
 'Warner Bros.',
 'Paramount Pictures',
 'Paramount Pictures',
 'Universal Pictures',
 '-',
 'Warner Bros.',
 'Warner Bros.',
 'Warner Bros.',
 '-',
 '-',
 'Twentieth Century Fox',
 '-',
 '-',
 'Universal Pictures',
 'Universal Pictures',
 '-',
 '-',
 '-',
 'Universal Pictures',
 '-',
 '-',
 'Warner Bros.',
 '-',
 'Warner Bros.',
 'Universal Pictures',
 'Universal Pictures',
 'Warner Bros.',
 '-',
 '-',
 'Twentieth Century Fox',
 '-',
 'Paramount Pictures',
 '-',
 'Warner Bros.',
 '-',
 'Warner Bros.',
 '-',
 '-',
 '-',
 'Warner Bros.',
 'Universal Pictures',
 'Warner Bros.',
 'Warner Bros.',
 'Paramount Pictures',
 '-',
 '-',
 '-',
 '-',
 'DreamWorks Distribution',
 '-',
 'Warner Bros.',
 '-',
 'Paramount Pictures',
 'Warner Bros.',
 'Lionsgate',
 '-',
 'Warner Bros.',
 '-',
 '-',
 'Lionsgate',
 'Warner Bros.',
 '-',
 '-',
 'Universal Pictures',
 

In [169]:
#A second check to make sure all lists are the same length
len(year_released) == len(max_theater_list) == len(theater_titles_list) == len(total_dom_list) == len(studio_list)

True

Now we will create a function to scrape from BoxOfficeMojo. This function will take the number of pages you would like to scrape from this website.  The function starts by creating empty lists which will eventually represent the columns we make in the dataframe.  The for loop will repeat each of the steps above for each webpage. The function will construct the dataframe and clean columns by removing specified characters and converting strings to integers. 

In [175]:
movies_in_theaters_df = pd.DataFrame()
def scraping_theaters(no_of_pages):
    """This function takes in an argument where you can select how many pages of data you would like to scrape
       from the Box Office Mojo - Most Theaters by Studio Type website. no_of_pages argument must be an integer
       greater than 0."""
    
    mojo_titles = []
    max_theaters_list = []
    year_released = []
    total_dom_list = []
    studios = []
    
    offset_value = [0,200,400,600,800] #variable to used as a part of the web address to select a webpage
    
    for value in offset_value[:no_of_pages]:
        url = f'https://www.boxofficemojo.com/chart/most_theaters/?offset={value}&by_studio_type=major'
        theaters_page = requests.get(url)
        theater_soup = BeautifulSoup(theaters_page.content,'lxml')
        
        mojo_container = theater_soup.find_all("td", class_="a-text-left mojo-field-type-release mojo-cell-wide")
        mojo_titles_collect = [mojo_container[i].findChild() for i in range(0, len(mojo_container))]
        mojo_titles += [mojo_titles_collect[x].text for x in range(0, len(mojo_titles_collect))]
        
        max_theaters = theater_soup.find_all("td", class_="a-text-right mojo-field-type-positive_integer")
        max_theaters_list += [max_theaters[i].text for i in range(0, len(max_theaters))]
        
        yr_container = theater_soup.find_all("td", class_="a-text-left mojo-field-type-date_interval")
        years = [yr_container[i].findChild() for i in range(0, len(yr_container))]
        year_released += [years[i].text[-4:] for i in range(0,len(years))]
        
        
        total_dom_gross = theater_soup.find_all("td", class_="a-text-right mojo-field-type-money")
        total_dom_list += [total_dom_gross[i].text for i in range(0,len(total_dom_gross))]
        
        studio_collect = theater_soup.find_all("td", class_="a-text-left mojo-field-type-release_studios")
        studios += [studio_collect[i].text.strip() for i in range(0, len(studio_collect))]
        
        time.sleep(0.3)
        
    df = pd.DataFrame([mojo_titles, max_theaters_list, year_released, total_dom_list, studios]).transpose()
    df.rename(columns={0:'title', 1:'max_theaters', 2:'year', 3:'total_dom_gross($)', 4:'studio'}, 
              inplace=True)
    
    df['max_theaters'] = df['max_theaters'].str.replace(',','').astype(int)
    df['year'] = df['year'].astype(int)
    df['total_dom_gross($)'] = df['total_dom_gross($)'].str.replace(',','').str.replace('$','').astype(int)
    
    return df
        
        
        
        
        
    
    
    

In [176]:
movies_in_theaters_df = scraping_theaters(5)

In [177]:
movies_in_theaters_df.head()

,title,max_theaters,year,total_dom_gross($),studio
0,The Lion King,4802,2019,543638043,-
1,Avengers: Endgame,4662,2019,858373000,-
2,Spider-Man: Far from Home,4634,2019,390532085,-
3,Toy Story 4,4575,2019,434038008,-
4,It Chapter Two,4570,2019,211593228,Warner Bros.


The movie theater data is almost clean.  It seems there are several movies where the studio is missing and has been replaced with a dash.

In [178]:
movies_in_theaters_df.loc[movies_in_theaters_df.studio == '-']

,title,max_theaters,year,total_dom_gross($),studio
0,The Lion King,4802,2019,543638043,-
1,Avengers: Endgame,4662,2019,858373000,-
2,Spider-Man: Far from Home,4634,2019,390532085,-
3,Toy Story 4,4575,2019,434038008,-
8,Aladdin,4476,2019,355559216,-
9,Avengers: Infinity War,4474,2018,678815482,-
10,Frozen II,4440,2019,477373578,-
11,Incredibles 2,4410,2018,608581744,-
12,Star Wars: Episode IX - The Rise of Skywalker,4406,2019,515202542,-
17,Solo: A Star Wars Story,4381,2018,213767512,-


In [179]:
len(movies_in_theaters_df.loc[movies_in_theaters_df.studio == '-'])

282

282 rows out of 1000 is 28.2% of our data. That's too much valuable information to drop.  It's also important to note that BoxOfficeMojo has classified these scraped movies as movies that were made by a major studio.  It's likely that a major studio has produced movies that have won awards or been very profitable.  We should find a way to populate the studio name for rows in which it is missing.

In [180]:
pd.set_option('display.max_rows', 500)
movies_in_theaters_df.loc[movies_in_theaters_df.studio == '-']

,title,max_theaters,year,total_dom_gross($),studio
0,The Lion King,4802,2019,543638043,-
1,Avengers: Endgame,4662,2019,858373000,-
2,Spider-Man: Far from Home,4634,2019,390532085,-
3,Toy Story 4,4575,2019,434038008,-
8,Aladdin,4476,2019,355559216,-
9,Avengers: Infinity War,4474,2018,678815482,-
10,Frozen II,4440,2019,477373578,-
11,Incredibles 2,4410,2018,608581744,-
12,Star Wars: Episode IX - The Rise of Skywalker,4406,2019,515202542,-
17,Solo: A Star Wars Story,4381,2018,213767512,-


By examining the BoxOffice Mojo website, we know we are missing two studios: Disney and Sony. This means that the missing studios will need to be sorted into two groups. I'll go through the rows of the dataframe that we printed above and store the indices associated with one of the two studios in a list.


In [181]:
#We can create different lists for the various missing studios and pull the corresponding indices.
#Then we'll use those lists in a for loop to populate the correct name.
disney_rows = [0,1,3,8,9,10,11,12,17,21,22,25,32,33,35,40,41,45,47,49,57,59,62,64,70,71,74,75,
              82,87,88,94,102,103,106,107,126,128,131,138,140,143,146,148,156,160,161,162,164,166,
              177,180,185,188,193,195,205,208,213,214,218,222,248,255,256,261,264,265,270,273,277,278,284,
              298,302,307,308,323,329,334,337,347,368,371,420,438,444,454,463,478,483,487,498,499,502,511,
              513,514,520,536,543,575,583,599,600,635,637,641,643,653,658,663,670,672,677,683,702,719,734,
              777,780,799,811,813,821,822,838,840,843,845,854,857,871,876,881,898,904,906,914,938,941,945,
              964,967,977,980,999]

In [182]:
len(disney_rows)

147

In [183]:
#Let's check to make sure all the studio is a dash for all these indices. 
#We don't want to accidentally overwrite data.
for row in disney_rows:
    if movies_in_theaters_df['studio'].loc[row] == '-':
        print(True)
    else:
        print('Double check your list. One of the numbers is wrong!')
        print(row)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


Great! We got `True` for every element in the `disney_rows` list.  That gives us confidence that we picked the right numbers and won't overwrite any data.

In [184]:
#Let's populate Disney as the studio and then filter by Disney to do a final check.
for row in disney_rows:
    movies_in_theaters_df['studio'].loc[row] = 'Disney'   

/Users/jeremylee/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [185]:
movies_in_theaters_df.loc[movies_in_theaters_df['studio'] == 'Disney']

,title,max_theaters,year,total_dom_gross($),studio
0,The Lion King,4802,2019,543638043,Disney
1,Avengers: Endgame,4662,2019,858373000,Disney
3,Toy Story 4,4575,2019,434038008,Disney
8,Aladdin,4476,2019,355559216,Disney
9,Avengers: Infinity War,4474,2018,678815482,Disney
10,Frozen II,4440,2019,477373578,Disney
11,Incredibles 2,4410,2018,608581744,Disney
12,Star Wars: Episode IX - The Rise of Skywalker,4406,2019,515202542,Disney
17,Solo: A Star Wars Story,4381,2018,213767512,Disney
21,Pirates of the Caribbean: At World's End,4362,2007,309420425,Disney


In [186]:
#The remaining movies should all be Sony. Let's preview those movies that still have a dash for studio name.
movies_in_theaters_df.loc[movies_in_theaters_df['studio'] == '-']

,title,max_theaters,year,total_dom_gross($),studio
2,Spider-Man: Far from Home,4634,2019,390532085,-
24,Spider-Man: Homecoming,4348,2017,334201140,-
28,Spider-Man 3,4324,2007,336530303,-
29,The Amazing Spider-Man 2,4324,2014,202853933,-
30,The Amazing Spider-Man,4318,2012,262030663,-
43,Hotel Transylvania 3: Summer Vacation,4267,2018,167510016,-
50,Venom,4250,2018,213515506,-
51,Men in Black 3,4248,2012,179020854,-
58,Jumanji: The Next Level,4227,2019,316831246,-
60,Men in Black: International,4224,2019,80001807,-


In [187]:
movies_in_theaters_df.loc[movies_in_theaters_df['studio'] == '-', 'studio'] = 'Sony'

In [188]:
movies_in_theaters_df['studio'].value_counts()
#Let's look at how many movies are associated with each studio

Warner Bros.                 208
Twentieth Century Fox        165
Disney                       147
Universal Pictures           136
Sony                         135
Paramount Pictures           112
Lionsgate                     49
DreamWorks Distribution       19
Metro-Goldwyn-Mayer (MGM)     14
TriStar Pictures               7
New Line Cinema                7
United Artists                 1
Name: studio, dtype: int64

In [474]:
#Let's save the cleaned dataframe to a csv file
movies_in_theaters_df.to_csv('movie_theater_data.csv')

## Scraping Award Data

We will be scraping the movie awards data from a Wikipedia webpage.

In [21]:
#We are only scraping one webpage for the awards dataframe. We won't need to write a function in this case.
award_page = requests.get('https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films')
award_soup = BeautifulSoup(award_page.content, 'lxml')

In [22]:
award_container = award_soup.find_all("td")
#All of the information we need lives inside <td> tags.

In [23]:
award_container

[<td><i><b><a href="/wiki/Parasite_(2019_film)" title="Parasite (2019 film)">Parasite</a></b></i></td>,
 <td><a href="/wiki/2019_in_film" title="2019 in film">2019</a></td>,
 <td>4</td>,
 <td>6
 </td>,
 <td><i><a href="/wiki/Ford_v_Ferrari" title="Ford v Ferrari">Ford v Ferrari</a></i></td>,
 <td><a href="/wiki/2019_in_film" title="2019 in film">2019</a></td>,
 <td>2</td>,
 <td>4
 </td>,
 <td><i><a href="/wiki/Learning_to_Skateboard_in_a_Warzone_(If_You%27re_a_Girl)" title="Learning to Skateboard in a Warzone (If You're a Girl)">Learning to Skateboard in a Warzone (If You're a Girl)</a></i></td>,
 <td><a href="/wiki/2019_in_film" title="2019 in film">2019</a></td>,
 <td>1</td>,
 <td>1
 </td>,
 <td><i><a href="/wiki/The_Neighbors%27_Window" title="The Neighbors' Window">The Neighbors' Window</a></i></td>,
 <td><a href="/wiki/2019_in_film" title="2019 in film">2019</a></td>,
 <td>1</td>,
 <td>1
 </td>,
 <td><i><a href="/wiki/Little_Women_(2019_film)" title="Little Women (2019 film)">Litt

We can see in `award_container` that the data we want for the first movies are in the first four `<td>` tags.  The data for the second movie are in the next 4 `<td>` tags. We can take advantage of that pattern to store that the title, year, nominations, and wins into four separate variables.

In [24]:
film_name_tags = award_container[0::4]
film_year_tags = award_container[1::4]
film_won_tags = award_container[2::4]
film_nom_tags = award_container[3::4]

In [25]:
#Use list comprehension to store information in four separate list variables
film_name = [film_name_tags[i].text for i in range(0,len(film_name_tags))]
film_year = [film_year_tags[i].text[0:4] for i in range(0,len(film_year_tags))]
awards_won = [film_won_tags[i].text for i in range(0, len(film_won_tags))]
awards_nom = [film_nom_tags[i].text.strip() for i in range(0,len(film_nom_tags))]

In [26]:
#Build the dataframe and rename the columns
awards_df = pd.DataFrame([film_name, film_year, awards_won, awards_nom]).transpose()
awards_df.rename(columns={0:'film_name', 1:'film_year', 2:'awards_won',3:'awards_nominated'}, inplace=True)

In [27]:
awards_df

,film_name,film_year,awards_won,awards_nominated
0,Parasite,2019,4,6
1,Ford v Ferrari,2019,2,4
2,Learning to Skateboard in a Warzone (If You're...,2019,1,1
3,The Neighbors' Window,2019,1,1
4,Little Women,2019,1,6
...,...,...,...,...
1314,You Can't Take It with You,1938,2,7
1315,Zorba the Greek (Alexis Zorbas),1964,3,7
1316,\nAcademy of Motion Picture Arts and Sciences ...,\nSup,\nEGOT\nTriple Crown of Acting\n,All nominees\nBest Actor (by age)\nBest Actres...
1317,\nBest Director (by age)\n,\nFil,\nAfrica\nArgentina\nAsia\nAustralia\nAustria\...,Asian\nBlack\nHispanic\nIndigenous\nJewish\nMu...


In [28]:
awards_df.drop(awards_df.index[1316:], inplace=True)
#the last 3 rows are not related to movies and can be dropped

In [29]:
awards_df

,film_name,film_year,awards_won,awards_nominated
0,Parasite,2019,4,6
1,Ford v Ferrari,2019,2,4
2,Learning to Skateboard in a Warzone (If You're...,2019,1,1
3,The Neighbors' Window,2019,1,1
4,Little Women,2019,1,6
...,...,...,...,...
1311,The Yankee Doodle Mouse,1943,1,1
1312,The Yearling,1946,2,7
1313,"Yesterday, Today and Tomorrow (Ieri, oggi, dom...",1964,1,1
1314,You Can't Take It with You,1938,2,7


The webpage has some unorthodox notation in the `awards_won` column.  Numbers in parentheses represent honorary awards.  We are only concerned with the awards won versus awards nominated. Let's look at an example:

In [30]:
awards_df.loc[awards_df['film_name'] =='Toy Story']

,film_name,film_year,awards_won,awards_nominated
364,Toy Story,1995,0 (1),3


In [31]:
awards_df['awards_won'] = awards_df['awards_won'].map(lambda x: x[0])
#let's grab the first element of the string

In [32]:
#Check to see that it worked
awards_df.loc[awards_df['film_name'] =='Toy Story']

,film_name,film_year,awards_won,awards_nominated
364,Toy Story,1995,0,3


In [33]:
#convert awards_won to integers
awards_df['awards_won'] = awards_df['awards_won'].astype(int)

In [34]:
#Three movies in the list won 10 or more awards so let's correct those rows
awards_df.loc[(awards_df['film_name'] == 'Titanic') & (awards_df['film_year'] == '1997'), 'awards_won'] = 11
awards_df.loc[awards_df['film_name'] == 'The Lord of the Rings: The Return of the King', 'awards_won'] = 11
awards_df.loc[awards_df['film_name'] == 'West Side Story', 'awards_won'] = 10

In [37]:
#awards_nominated has a few rows we will need to clean up as well
#our code also captured a footnote on some of the rows
awards_df.loc[awards_df['film_name'] =='Sunrise']

,film_name,film_year,awards_won,awards_nominated
1215,Sunrise,1927,3,4[10]


In [38]:
#10 films where awards_nominated needed to be cleaned up 

awards_df.loc[awards_df['film_name'] == 'Just Another Missing Kid', 'awards_nominated'] = '1'
awards_df.loc[awards_df['film_name'] == 'Amarcord', 'awards_nominated'] = '3'
awards_df.loc[awards_df['film_name'] == 'The Godfather', 'awards_nominated'] = '10' 
awards_df.loc[awards_df['film_name'] == 'A Christmas Carol', 'awards_nominated'] = '1'
awards_df.loc[awards_df['film_name'] == 'Limelight', 'awards_nominated'] = '1'
awards_df.loc[awards_df['film_name'] == "A Midsummer Night's Dream", 'awards_nominated'] = '4'
awards_df.loc[awards_df['film_name'] == 'An Occurrence at Owl Creek Bridge', 'awards_nominated'] = '1'
awards_df.loc[awards_df['film_name'] == 'The Rains Came', 'awards_nominated'] = '6'
awards_df.loc[awards_df['film_name'] == 'Sunrise', 'awards_nominated'] = '4'
awards_df.loc[awards_df['film_name'] == 'The Wizard of Oz', 'awards_nominated'] = '6'

awards_df.loc[awards_df['film_name'] =='Sunrise']

,film_name,film_year,awards_won,awards_nominated
1215,Sunrise,1927,3,4


In [39]:
awards_df['awards_nominated'] = awards_df['awards_nominated'].astype(int)

In [40]:
#create a new column called win_rate which is equal to awards_won/awards_nominated
awards_df['win_rate'] = round(awards_df['awards_won'] / awards_df['awards_nominated'], 3)

In [41]:
awards_df.head(20)

,film_name,film_year,awards_won,awards_nominated,win_rate
0,Parasite,2019,4,6,0.667
1,Ford v Ferrari,2019,2,4,0.500
2,Learning to Skateboard in a Warzone (If You're...,2019,1,1,1.000
3,The Neighbors' Window,2019,1,1,1.000
4,Little Women,2019,1,6,0.167
5,Marriage Story,2019,1,6,0.167
6,Jojo Rabbit,2019,1,6,0.167
7,Toy Story 4,2019,1,2,0.500
8,Joker,2019,2,11,0.182
9,Once Upon a Time in Hollywood,2019,2,10,0.200


In [42]:
#convert years to integer type
awards_df['film_year'] = awards_df['film_year'].astype(int)

In [43]:
awards_df.head(20)

,film_name,film_year,awards_won,awards_nominated,win_rate
0,Parasite,2019,4,6,0.667
1,Ford v Ferrari,2019,2,4,0.500
2,Learning to Skateboard in a Warzone (If You're...,2019,1,1,1.000
3,The Neighbors' Window,2019,1,1,1.000
4,Little Women,2019,1,6,0.167
5,Marriage Story,2019,1,6,0.167
6,Jojo Rabbit,2019,1,6,0.167
7,Toy Story 4,2019,1,2,0.500
8,Joker,2019,2,11,0.182
9,Once Upon a Time in Hollywood,2019,2,10,0.200


In [44]:
#Clean up the film titles 
awards_df['film_name'] = awards_df['film_name'].map(lambda x: x.strip())

In [45]:
awards_df.head(20)

,film_name,film_year,awards_won,awards_nominated,win_rate
0,Parasite,2019,4,6,0.667
1,Ford v Ferrari,2019,2,4,0.500
2,Learning to Skateboard in a Warzone (If You're...,2019,1,1,1.000
3,The Neighbors' Window,2019,1,1,1.000
4,Little Women,2019,1,6,0.167
5,Marriage Story,2019,1,6,0.167
6,Jojo Rabbit,2019,1,6,0.167
7,Toy Story 4,2019,1,2,0.500
8,Joker,2019,2,11,0.182
9,Once Upon a Time in Hollywood,2019,2,10,0.200


In [46]:
#save as a csv file
awards_df.to_csv('movie_awards.csv')

Now we've got three clean dataframes saved as csv files which we will import and use for analysis in a separate notebook.